# Correlation matrix and visualization of variables and GDP

### This is the master notebook, please copy and create a new one for each group.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro

In [12]:
df= pd.read_csv (os.getcwd()+'/WDI/'+'Europe.csv')
df

,Unnamed: 0,Country,Year,Alcohol per capita,Education GExp,Employment-agriculture,Employment-industry,Employment-services,Exports-Commercial services,Exports-G&S,...,Literacy rate,Mortality-infants,Mortality-pollution,Net migration,Ninis,R&D GExp,Renewable electricity,Suicide,Workers high education,Continent
0,347,DEU,1990,1.000000,0.443529,1.000000,1.000000,0.000000,0.002386,0.000000,...,NaN,1.000000,NaN,0.966124,0.777580,0.000000,0.000000,0.956522,1.000000,Europe
1,348,DEU,1991,1.000000,0.443529,1.000000,1.000000,0.000000,0.000000,0.025535,...,NaN,1.000000,NaN,0.966124,0.777580,0.000000,0.002525,0.956522,1.000000,Europe
2,349,DEU,1992,1.000000,0.443529,0.964758,0.966793,0.034321,0.018415,0.046400,...,NaN,0.875324,NaN,0.966124,0.777580,0.000000,0.004025,0.956522,1.000000,Europe
3,350,DEU,1993,1.000000,0.443529,0.942731,0.907021,0.088143,0.012182,0.010957,...,NaN,0.765220,NaN,0.823187,0.777580,0.000000,0.005848,0.956522,1.000000,Europe
4,351,DEU,1994,1.000000,0.380220,0.903084,0.876660,0.119345,0.018513,0.041261,...,NaN,0.670984,NaN,0.680250,0.777580,0.000000,0.011012,0.956522,1.000000,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,1000,NLD,2016,0.013072,0.968750,0.082949,0.041295,0.949686,0.740369,0.749611,...,NaN,0.000000,NaN,0.111948,0.379310,0.691544,1.000000,0.945946,0.357541,Europe
309,1001,NLD,2017,0.026144,0.854247,0.087558,0.045759,0.946092,0.842405,0.869072,...,NaN,0.000000,NaN,0.139935,0.051724,0.666094,1.000000,1.000000,0.407823,Europe
310,1002,NLD,2018,0.039216,1.000000,0.009217,0.006696,0.992813,0.993077,1.000000,...,NaN,0.007133,NaN,0.139935,0.155172,1.000000,1.000000,0.810811,0.670389,Europe
311,1003,NLD,2019,0.039216,1.000000,0.000000,0.000000,1.000000,1.000000,0.962069,...,NaN,0.017118,NaN,0.139935,0.201150,1.000000,1.000000,0.918919,0.547488,Europe


## Visualization

In [13]:
import plotly.express as px

In [14]:
myvariables=['Alcohol per capita',

 'Education GExp',

 'Employment-agriculture',

 'Employment-industry',

 'Employment-services',

 'Exports-Commercial services',

 'Exports-G&S',

 'Fertility rate',

 'Foreign investment',

 'GDP',

 'Health services use',

 'International taxes',

 'Literacy rate',

 'Mortality-infants',

 'Net migration',

 'Ninis',

 'R&D GExp',

 'Renewable electricity',

 'Suicide',

 'Workers high education']
europe_list=['DEU','FRA','SWE','GBR','ESP','HRV','POL','GRC','AUT','NLD']

In [15]:
for i in range(0,len(europe_list)):

    dat=df.loc[df.loc[:, 'Country'] == europe_list[i]]

    a=px.line(dat, x="Year", y=myvariables, width=800, height=600,title=europe_list[i])

    display(a)

In [16]:
columns=['Country','Year','Gender equality','Exports-Commercial services','Renewable electricity','Employment-agriculture','Employment-industry','Employment-services','Exports-G&S','Fertility rate','Foreign investment','GDP','Education GExp','Workers high education','Literacy rate','Mortality-pollution','Net migration','Mortality-infants','Health services use','R&D GExp','Ninis','Suicide','International taxes','Alcohol per capita']
clist=df['Country'].unique()

The following loop has been created to test if each variable for each country follows a normal distribution and also to present in an orderly manner all the correlations between the GDP and the variables so they can be understood on a lower scale.

In [17]:
def multcolumn(frame):
    for u in range(3, len(columns)):
        name2=columns[u]+'.^2'
        name3=columns[u]+'.^3'
        namelog=columns[u]+'.log'
        frame.loc[:,name2] = frame[columns[u]]**2
        frame.loc[:,name3] = frame[columns[u]]**3
        frame.loc[:,namelog] = np.log(frame[columns[u]])

In [18]:
multcolumn(df)
for i in range(0,len(clist)):
    dat=df.loc[df.loc[:, 'Country'] == clist[i]]
    for e in range(2,len(columns)):
        data=dat.iloc[:, e]
        stat, p = shapiro(data)
        print(clist[i] +"-"+ columns[e])
        print('Statistical=%.3f, p=%.3f' % (stat, p))
        alpha = 0.05
        if p > alpha:
            print('Data is NORMAL ( H0 not denied )')
        else:
            print('It does NOT seem normal( Denies H0 )')
    print(clist[i])
    cor=dat.corr()    
    cor.loc[:,'GDP-R^2'] = cor['GDP']**2
    cor.loc[:,'Indicator']=cor.index
    cor[['Indicator','Type']]=cor.Indicator.str.split('.',expand=True)  
    corcolumn=cor[['Indicator','Type','GDP-R^2','GDP']]
    corcolumn=corcolumn.loc[corcolumn.loc[:, 'GDP-R^2'] >= 0.75]
    id=corcolumn.groupby('Indicator')['GDP-R^2'].transform(max)==corcolumn['GDP-R^2']
    corcolumn[id] 
    max_df=pd.DataFrame(corcolumn[id])
    max_df['Behaviour']=np.where(max_df['GDP']>0, 'Positive', 'Negative')
    max_df['Type']=max_df['Type'].replace(['^2','^3','log'],['Cuadratic','Cubic','Logarithmic'])
    max_df.drop("GDP",axis=1,inplace=True)
    max_df=max_df.reset_index(drop=True)
    max_df = max_df.drop(max_df[max_df['Indicator']=='Year'].index)
    max_df = max_df.drop(max_df[max_df['Indicator']=='GDP'].index)
    max_df = max_df.drop(max_df[max_df['Indicator']=='Unnamed: 0'].index)
    max_df=max_df.sort_values(by = 'GDP-R^2',ascending = False).style.background_gradient()
    display(max_df)

DEU-Gender equality
Statistical=0.957, p=0.229
Data is NORMAL ( H0 not denied )
DEU-Exports-Commercial services
Statistical=0.806, p=0.000
It does NOT seem normal( Denies H0 )
DEU-Renewable electricity
Statistical=0.888, p=0.003
It does NOT seem normal( Denies H0 )
DEU-Employment-agriculture
Statistical=0.906, p=0.009
It does NOT seem normal( Denies H0 )
DEU-Employment-industry
Statistical=0.877, p=0.002
It does NOT seem normal( Denies H0 )
DEU-Employment-services
Statistical=0.887, p=0.003
It does NOT seem normal( Denies H0 )
DEU-Exports-G&S
Statistical=0.893, p=0.004
It does NOT seem normal( Denies H0 )
DEU-Fertility rate
Statistical=0.851, p=0.000
It does NOT seem normal( Denies H0 )
DEU-Foreign investment
Statistical=0.927, p=0.031
It does NOT seem normal( Denies H0 )
DEU-GDP
Statistical=0.972, p=0.548
Data is NORMAL ( H0 not denied )
DEU-Education GExp
Statistical=0.887, p=0.003
It does NOT seem normal( Denies H0 )
DEU-Workers high education
Statistical=nan, p=1.000
Data is NORMAL

c:\Users\msanchis\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log

c:\Users\msanchis\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log

c:\Users\msanchis\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log

c:\Users\msanchis\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log

c:\Users\msanchis\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log

c:\Users\msanchis\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero 

,Indicator,Type,GDP-R^2,Behaviour
5,Exports-G&S,None,0.954928,Positive
7,Health services use,None,0.916594,Positive
4,Exports-Commercial services,None,0.911725,Positive
10,Employment-services,Cuadratic,0.883525,Positive
12,Alcohol per capita,Cubic,0.875820,Negative
11,Suicide,Cubic,0.871691,Negative
2,Employment-agriculture,None,0.855832,Negative
3,Employment-industry,None,0.828837,Negative
9,Renewable electricity,None,0.814635,Positive
8,R&D GExp,None,0.790969,Positive


FRA-Gender equality
Statistical=0.957, p=0.229
Data is NORMAL ( H0 not denied )
FRA-Exports-Commercial services
Statistical=0.796, p=0.000
It does NOT seem normal( Denies H0 )
FRA-Renewable electricity
Statistical=0.936, p=0.058
Data is NORMAL ( H0 not denied )
FRA-Employment-agriculture
Statistical=0.904, p=0.008
It does NOT seem normal( Denies H0 )
FRA-Employment-industry
Statistical=0.906, p=0.009
It does NOT seem normal( Denies H0 )
FRA-Employment-services
Statistical=0.915, p=0.015
It does NOT seem normal( Denies H0 )
FRA-Exports-G&S
Statistical=0.876, p=0.002
It does NOT seem normal( Denies H0 )
FRA-Fertility rate
Statistical=0.884, p=0.002
It does NOT seem normal( Denies H0 )
FRA-Foreign investment
Statistical=0.915, p=0.015
It does NOT seem normal( Denies H0 )
FRA-GDP
Statistical=0.957, p=0.223
Data is NORMAL ( H0 not denied )
FRA-Education GExp
Statistical=0.846, p=0.000
It does NOT seem normal( Denies H0 )
FRA-Workers high education
Statistical=nan, p=1.000
Data is NORMAL ( H

,Indicator,Type,GDP-R^2,Behaviour
5,Exports-G&S,None,0.959796,Positive
2,Alcohol per capita,None,0.938995,Negative
11,Health services use,Cubic,0.920650,Negative
7,Exports-Commercial services,Logarithmic,0.865167,Positive
4,Employment-services,None,0.848399,Positive
12,Suicide,Cubic,0.847857,Negative
9,Employment-industry,Cuadratic,0.842783,Negative
8,Renewable electricity,Logarithmic,0.839625,Positive
3,Employment-agriculture,None,0.821163,Negative
10,Mortality-infants,Logarithmic,0.770045,Negative


...

,Indicator,Type,GDP-R^2,Behaviour
2,Net migration,Logarithmic,0.946387,Negative
0,Exports-Commercial services,None,0.769393,Positive


AUT-Gender equality
Statistical=0.957, p=0.229
Data is NORMAL ( H0 not denied )
AUT-Exports-Commercial services
Statistical=0.807, p=0.000
It does NOT seem normal( Denies H0 )
AUT-Renewable electricity
Statistical=0.979, p=0.780
Data is NORMAL ( H0 not denied )
AUT-Employment-agriculture
Statistical=0.935, p=0.053
Data is NORMAL ( H0 not denied )
AUT-Employment-industry
Statistical=0.882, p=0.002
It does NOT seem normal( Denies H0 )
AUT-Employment-services
Statistical=0.897, p=0.005
It does NOT seem normal( Denies H0 )
AUT-Exports-G&S
Statistical=0.791, p=0.000
It does NOT seem normal( Denies H0 )
AUT-Fertility rate
Statistical=0.856, p=0.001
It does NOT seem normal( Denies H0 )
AUT-Foreign investment
Statistical=0.980, p=0.804
Data is NORMAL ( H0 not denied )
AUT-GDP
Statistical=0.793, p=0.000
It does NOT seem normal( Denies H0 )
AUT-Education GExp
Statistical=0.854, p=0.000
It does NOT seem normal( Denies H0 )
AUT-Workers high education
Statistical=nan, p=1.000
Data is NORMAL ( H0 no

,Indicator,Type,GDP-R^2,Behaviour
4,Exports-G&S,None,0.981555,Positive
10,Health services use,Cubic,0.954586,Negative
12,Alcohol per capita,Cuadratic,0.953465,Negative
11,Suicide,Cuadratic,0.940341,Negative
8,Renewable electricity,Logarithmic,0.888040,Positive
6,R&D GExp,None,0.886242,Positive
9,Employment-services,Cuadratic,0.883511,Positive
3,Employment-industry,None,0.829776,Negative
7,Exports-Commercial services,Logarithmic,0.772605,Positive
2,Employment-agriculture,None,0.767761,Negative


NLD-Gender equality
Statistical=0.957, p=0.247
Data is NORMAL ( H0 not denied )
NLD-Exports-Commercial services
Statistical=0.814, p=0.000
It does NOT seem normal( Denies H0 )
NLD-Renewable electricity
Statistical=0.945, p=0.115
Data is NORMAL ( H0 not denied )
NLD-Employment-agriculture
Statistical=0.935, p=0.059
Data is NORMAL ( H0 not denied )
NLD-Employment-industry
Statistical=0.910, p=0.013
It does NOT seem normal( Denies H0 )
NLD-Employment-services
Statistical=0.921, p=0.025
It does NOT seem normal( Denies H0 )
NLD-Exports-G&S
Statistical=0.895, p=0.005
It does NOT seem normal( Denies H0 )
NLD-Fertility rate
Statistical=0.874, p=0.002
It does NOT seem normal( Denies H0 )
NLD-Foreign investment
Statistical=0.939, p=0.076
Data is NORMAL ( H0 not denied )
NLD-GDP
Statistical=0.914, p=0.017
It does NOT seem normal( Denies H0 )
NLD-Education GExp
Statistical=0.863, p=0.001
It does NOT seem normal( Denies H0 )
NLD-Workers high education
Statistical=nan, p=1.000
Data is NORMAL ( H0 no

,Indicator,Type,GDP-R^2,Behaviour
4,Exports-G&S,None,0.955942,Positive
7,Net migration,None,0.945714,Negative
11,Alcohol per capita,Cubic,0.937530,Negative
10,Health services use,Cuadratic,0.936751,Negative
9,Exports-Commercial services,Logarithmic,0.915834,Positive
6,Mortality-infants,None,0.903385,Negative
8,Renewable electricity,None,0.885641,Positive
2,Employment-industry,None,0.852451,Negative
3,Employment-services,None,0.819158,Positive
